In [75]:
!pip install splinter

In [76]:
# Declare Dependencies 
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [77]:
!which chromedriver

/usr/local/bin/chromedriver


# NASA Mars News

In [78]:
# Mar User
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [79]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [80]:
# HTML Object
html = browser.html
# Create BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')

In [81]:
# Collect the latest News Title and Paragraph Text
news_title = soup.find('div', class_= 'content_title').find('a').text
news_p = soup.find('div', class_= 'article_teaser_body').text

print(news_title)
print(news_p)

For InSight, Dust Cleanings Will Yield New Science
Wind can be crucial to clearing dust from spacecraft solar panels on Mars. With InSight's meteorological sensors, scientists get their first measurements of wind and dust interacting "live" on the Martian surface.  


# JPL Mars Space Images - Featured Image

In [82]:
#Visit the url for Featured Space Image
url= 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

# Asking splinter to go to the site hit a button with id name full_image
# Find the image url for the current Featured Mars Image
full_image_button = browser.find_by_id("full_image")
full_image_button.click()


In [83]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_element = browser.find_link_by_partial_text('more info')
more_info_element.click()

In [84]:
# Parse the results html with soup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')

img_url = image_soup.select_one('figure.lede a img').get('src')
img_url

'/spaceimages/images/largesize/PIA19041_hires.jpg'

In [85]:
# Use the base url to create an absolute url
featured_image_url = f'https://www.jpl.nasa.gov{img_url}'
featured_image_url 

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19041_hires.jpg'

# Mars Weather

In [86]:
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [87]:
# HTML Object 
html = browser.html

# Parse HTML with Beautiful Soup
weather_soup = BeautifulSoup(html, 'html.parser')

# First find a tweet with the data-name 'Mars Weather'
mars_weather_tweet = weather_soup.find('div',
                                      attrs={"class":"tweet",
                                            "data-name":"Mars Weather"})

# Find all elements that contain tweets
mars_weather = mars_weather_tweet.find('p', 'tweet-text').get_text()
mars_weather

'InSight sol 156 (2019-05-05) low -99.2ºC (-146.6ºF) high -18.1ºC (-0.6ºF)\nwinds from the SW at 4.7 m/s (10.5 mph) gusting to 13.8 m/s (30.8 mph)\npressure at 7.40 hPapic.twitter.com/FlGDeYg8hi'

# Mars Facts

In [88]:
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [89]:
type(tables)

list

In [90]:
df = tables[0]

#Create Columns' names
df.columns = ['Description','Value']

#Reset index
df.set_index('Description', inplace=True)

# Save the table directly to a file
df.to_html('table.html')
df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


# Mars Hemispheres

In [91]:
# Mar User
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [103]:
# Visit hemispheres website through splinter module 
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(hemispheres_url)

hemisphere_image_urls = []

#First get a list of all the hemispheres
links = browser.find_by_css('a.product-item h3')

for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the element on each loop to avoid a stale element
    browser.find_by_css('a.product-item h3')[i].click()
    
    #Get Hemisphere title
    hemisphere['title'] = browser.find_by_css('h2.title').text
    
    #Next we fomd the Sample image anchor tage and extract the href
    sample_element = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_element['href']
    
    #Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    #Finally, we navigate back
    browser.back()
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]